In [1]:
import cobra
import cobra.medium
from cobra.io import load_json_model
from cobra import flux_analysis
import pandas as pd
import numpy as np
from cobra.flux_analysis import production_envelope
from cobra import Reaction
from cobra import Solution

## Avaliando o consumo de Glicose e Oxigênio

In [2]:
# reacao = modelobeta.reactions.get_by_id("R_O2xtI")
# modelobeta.reactions.R_O2xtI.compartments

modelopkdupla = load_json_model("ViaPKDupla2.json") # Malonil-CoA e Oxaloacetato
cobra.medium.boundary_types.find_external_compartment(modelopkdupla)
# cobra.flux_analysis.phenotype_phase_plane._find_carbon_sources(modelopkdupla)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


'C_e'

In [3]:
producao = production_envelope(
    modelopkdupla, ["R_O2xtI"], objective="R_3HPxtO", carbon_sources="R_GLCxtI")

In [4]:
# display(producao.to_string())
producao.head()

# producao.plot(kind = 'line', x='R_O2xtI', y='carbon_yield_maximum')

,carbon_source,flux_minimum,carbon_yield_minimum,mass_yield_minimum,flux_maximum,carbon_yield_maximum,mass_yield_maximum,R_O2xtI
0,R_GLCxtI,0.0,NaN,NaN,18.43038,NaN,NaN,-1000.000000
1,R_GLCxtI,0.0,NaN,NaN,18.43038,NaN,NaN,-947.368421
2,R_GLCxtI,0.0,NaN,NaN,18.43038,NaN,NaN,-894.736842
3,R_GLCxtI,0.0,NaN,NaN,18.43038,NaN,NaN,-842.105263
4,R_GLCxtI,0.0,NaN,NaN,18.43038,NaN,NaN,-789.473684


## Simulação por MEWpy usando FBA

In [5]:
solucao = modelopkdupla.optimize(objective_sense="maximize")
solucao
#solucao.to_csv("C:\\Users\\tiago220053\\Desktop\\GraficosModelos\\Solucao_PKDupla.csv")

,fluxes,reduced_costs
R_PDE1,0.000000e+00,0.000000e+00
R_AGP3_2,0.000000e+00,-1.053593e-16
R_AGP3_1,0.000000e+00,0.000000e+00
R_GUA1,4.501364e-14,-3.108624e-15
R_PHO84,4.241828e-14,0.000000e+00
...,...,...
R_HOM6_2,0.000000e+00,-2.025316e-01
R_CITxtI,0.000000e+00,-2.848101e+00
R_APA1_2,0.000000e+00,1.068590e-15
R_GLTxtO,0.000000e+00,0.000000e+00


## Encontrando reações essenciais

In [18]:
cobra.flux_analysis.find_essential_reactions(modelopkdupla)

{<Reaction R_1002_ at 0x185251e76a0>,
 <Reaction R_3HPxtO at 0x18524652550>,
 <Reaction R_AAT2_1 at 0x18520c37820>,
 <Reaction R_ACC1 at 0x18522c9b280>,
 <Reaction R_ALD6 at 0x18524100100>,
 <Reaction R_ATP1 at 0x18523d95d60>,
 <Reaction R_CO2xtO at 0x18523c01fa0>,
 <Reaction R_COX1 at 0x1852434d820>,
 <Reaction R_CaMCRN at 0x18522cbf190>,
 <Reaction R_FBA1 at 0x18523fe8d60>,
 <Reaction R_GLCxtI at 0x185244ffe20>,
 <Reaction R_GLN1 at 0x1851f12b550>,
 <Reaction R_GLT1 at 0x1851f30dfa0>,
 <Reaction R_GLY1 at 0x18522d72070>,
 <Reaction R_HOM2 at 0x1852180abb0>,
 <Reaction R_HOM3 at 0x1852115d2e0>,
 <Reaction R_HOM6_1 at 0x18525545760>,
 <Reaction R_IPP1 at 0x1851f169160>,
 <Reaction R_MCRC at 0x18522ef1550>,
 <Reaction R_O2xtI at 0x185210bb220>,
 <Reaction R_PGK1 at 0x18523bd15b0>,
 <Reaction R_PpMdIC at 0x18521f2ffa0>,
 <Reaction R_RIP1 at 0x18525227df0>,
 <Reaction R_RKI1 at 0x1852444f040>,
 <Reaction R_RPE1 at 0x185249aaaf0>,
 <Reaction R_THR1 at 0x185229038b0>,
 <Reaction R_THR4_1 at

## Nocaute por MEWpy

In [6]:
single = cobra.flux_analysis.single_gene_deletion(modelopkdupla)
double = cobra.flux_analysis.double_gene_deletion(modelopkdupla)

print(double.knockout[{"YOL086C", "YMR303C"}])

#modelopkdupla.reactions.knock_out()

                      ids    growth   status
49540  {YMR303C, YOL086C}  18.43038  optimal


## Outras sintaxes

In [6]:
#### Análises de fluxos

### Deleção
## cobra.flux_analysis.deletion
    ## _reaction_deletion(str) (Deleta reação)
    ## _gene_deletion(str) (Deleta gene)
    ## _multi_deletion(pd) (Multiplos nocautes)
    ## _elements_lists(list) (Lista de elementos)
    ## cobra.flux_analysis.single_gene_deletion (Nocauteia cada gene de uma gene_list)
    ## cobra.flux_analysis.double_gene_deletion (Nocauteia cada par de genes a partir da combinação de duas listas)
    ## cobra.flux_analysis.single_reaction_deletion (Nocauteia cada reação de uma reaction_list)
    ## cobra.flux_analysis.double_reaction_deletion (Nocauteia cada par de reações a partir da combinação de duas listas)

### Remoção de loops termodinamicamente inviáveis
## cobra.flux_analysis.loopless
    ## cobra.flux_analysis.add_loopless (Modifica um modelo para que todos as distribuições de fluxo viáveis não sejam loop)
    ## cobra.flux_analysis.loopless_solution (Converte uma solução existente para uma sem loop)
    ## cobra,flux_analysis.loopless_fva_iter(Uniao[float, Dict[str,float]]) (Solução Loopless FVA para iteração única)

## Adiciona restrições e objetivos para aplicar o MOMA (Minimization Of Metabolic Adjustment)
# cobra.flux_analysis.add_moma
    ## Computa uma solução única baseado na MOMA linear
    # cobra.flux_analysis.moma

## Adiciona restrições e objetivos para aplicar o ROOM (Regulatory on/off minimization)
# cobra.flux_analysis.add_room
    ## Computa uma solução baseada no ROOM
    # cobra.flux_analysis.room

### pFBA (Parsimonious Enzyme Usage Flux Balance Analysis)
## cobra.flux_analysis.parsimonious
    ## cobra.flux_analysis.optimize_minimal_flux(modelo) (PFBA basico para minimizar fluxo total)
    ## cobra.flux_analysis.pfba(solucao) (Realiza um pFBA (Parsimonious Enzyme Usage Flux Balance Analysis) básico, adicionando a minimização de todos os fluxos para o objetivo do modelo.)
    ## cobra.flux_analysis.add_pfba (Adiciona um objetivo pFBA para minimizar o fluxo de todas as reações do objetivo atual)

## Encontra reações que não podem carregar nenhum fluxo
# cobra.flux_analysis.find_blocked_reactions

## Retorna um set de todos os genes essenciais (essencial = zerar reações 
# dependentes desse gene = zerar o objetivo)
# cobra.flux_analysis.find_essential_genes

## Retorna um set de todas as reações essenciais (essencial = zerar seu fluxo 
# = zerar objetivo)
# cobra.flux_analysis.find_essential_reactions

## Determina o valor mínimo e máximo para cada reação (nem sempre resulta em solução
# viável)
# cobra.flux_analysis.flux_variability_analysis

# cobra.flux_analysis.variability

## FBA geométrica para obter uma distribuição de fluxo única e centrada
# cobra.flux_analysis.geometric_fba

# Nocaute:

# reaction.knock_out()